In [28]:
import pandas as pd
import numpy as np
file_path = r'D:\Pokemon.csv'
df = pd.read_csv(file_path, encoding='latin1')
print("数据集加载成功！")
print("原始数据形状:", df.shape)
df_cleaned = df.copy()

数据集加载成功！
原始数据形状: (810, 13)


In [29]:
# --- 问题一：删除最后两行 ---
print("处理前的数据形状:", df_cleaned.shape)

# 使用 .index 获取最后两行的索引，然后使用 .drop() 删除
# inplace=True 表示直接在原数据上修改
rows_to_drop = df_cleaned.tail(2).index
df_cleaned.drop(rows_to_drop, inplace=True)

print("处理后的数据形状:", df_cleaned.shape)
print("\n成功删除最后两行数据。")

处理前的数据形状: (810, 13)
处理后的数据形状: (808, 13)

成功删除最后两行数据。


In [26]:
# --- 问题二：将 'Type 2' 列中的缺失值(NaN)替换为'None'
print("处理前 'Type 2' 的值分布情况:")
# 使用 .value_counts() 可以查看每个类别出现的次数，NaN默认不显示
display(df_cleaned['Type 2'].value_counts())
nan_count_before = df_cleaned['Type 2'].isna().sum()
print(f"处理前共有 {nan_count_before} 个缺失值 (NaN)。")

df_cleaned['Type 2'].fillna('None', inplace=True)


print("\n----------------------------------")
print("处理后 'Type 2' 的值分布情况:")
# 再次查看值分布，现在 'None' 已经成为一个明确的类别
display(df_cleaned['Type 2'].value_counts())
nan_count_after = df_cleaned['Type 2'].isna().sum()
print(f"处理后共有 {nan_count_after} 个缺失值 (NaN)。")
print("\n成功将所有缺失的第二属性替换为 'None' 类别。")

处理前 'Type 2' 的值分布情况:


Flying       97
Ground       35
Poison       34
Psychic      33
Fighting     26
Grass        25
Fairy        23
Steel        22
Dark         20
Dragon       18
Ice          14
Water        14
Rock         14
Ghost        14
Fire         12
Electric      6
Normal        4
Bug           3
A             1
undefined     1
273           1
0             1
BBB           1
Name: Type 2, dtype: int64

处理前共有 383 个缺失值 (NaN)。

----------------------------------
处理后 'Type 2' 的值分布情况:


None         383
Flying        97
Ground        35
Poison        34
Psychic       33
Fighting      26
Grass         25
Fairy         23
Steel         22
Dark          20
Dragon        18
Ice           14
Water         14
Rock          14
Ghost         14
Fire          12
Electric       6
Normal         4
Bug            3
A              1
undefined      1
273            1
0              1
BBB            1
Name: Type 2, dtype: int64

处理后共有 0 个缺失值 (NaN)。

成功将所有缺失的第二属性替换为 'None' 类别。


In [30]:
# --- 问题三：删除重复行 ---

print("处理前的数据形状:", df_cleaned.shape)

# 计算有多少重复行
duplicate_count = df_cleaned.duplicated().sum()
print(f"发现 {duplicate_count} 条重复行。")

# 删除重复行
# keep='first' 表示保留第一次出现的记录，删除后面的重复项
df_cleaned.drop_duplicates(inplace=True, keep='first')

print("处理后的数据形状:", df_cleaned.shape)
print("\n成功删除所有重复行。")

处理前的数据形状: (808, 13)
发现 6 条重复行。
处理后的数据形状: (802, 13)

成功删除所有重复行。


In [24]:
# --- 问题四：处理 Attack 异常值 ---

# **【修正】** 在计算前，必须将'Attack'列转换为数值类型
#  我们使用 pd.to_numeric，errors='coerce' 会将无法转换的值设为缺失值(NaN)
df_cleaned['Attack'] = pd.to_numeric(df_cleaned['Attack'], errors='coerce')

# （可选步骤）如果存在因转换失败而产生的NaN，可以用中位数等填充
# df_cleaned['Attack'].fillna(df_cleaned['Attack'].median(), inplace=True)

# --- 现在，您原来的代码就可以正常运行了 ---

# 首先，查看Attack属性的描述性统计和最大值
print("处理前 'Attack' 属性的统计信息:")
print(df_cleaned['Attack'].describe())

# 找到最大值对应的宝可梦
max_attack_pokemon = df_cleaned.loc[df_cleaned['Attack'].idxmax()]
print("\n拥有最高 Attack 值的宝可梦:\n", max_attack_pokemon)

# 一种常见的处理方法是用该列的99%分位数来替换极端最大值
# 这样既修正了异常，又保留了数据的高端分布
ninety_ninth_percentile = df_cleaned['Attack'].quantile(0.99)
print(f"\n'Attack' 的99%分位数值为: {ninety_ninth_percentile}")

# 将所有超过这个值的 Attack 都设置为99%分位数的值
df_cleaned.loc[df_cleaned['Attack'] > ninety_ninth_percentile, 'Attack'] = ninety_ninth_percentile

print("\n处理后的 'Attack' 属性最大值:", df_cleaned['Attack'].max())
print("成功将过高的 Attack 值修正。")

处理前 'Attack' 属性的统计信息:
count     800.000000
mean       81.095000
std        53.245327
min         5.000000
25%        55.000000
50%        75.000000
75%       100.000000
max      1000.000000
Name: Attack, dtype: float64

拥有最高 Attack 值的宝可梦:
 #                128
Name          Tauros
Type 1        Normal
Type 2           NaN
Total            490
HP                75
Attack          1000
Defense           95
Sp. Atk           40
Sp. Def           70
Speed            110
Generation         1
Legendary      FALSE
Name: 140, dtype: object

'Attack' 的99%分位数值为: 170.0

处理后的 'Attack' 属性最大值: 170.0
成功将过高的 Attack 值修正。


In [25]:
# --- 问题五：修正 Generation 和 Legendary 置换 ---

# 正常情况下，'Generation' 应该是数字，'Legendary' 应该是布尔值 (True/False)
# 我们需要找到 'Generation' 列的值是布尔值的行
# Series.apply(type) is slow, a better way is to check the type of the first element
# However, for simplicity and clarity in finding mixed types, we can do this:
try:
    # 尝试将 'Generation' 列转换为数字，无法转换的将变成 NaT/NaN
    gen_as_numeric = pd.to_numeric(df_cleaned['Generation'], errors='coerce')
    # 找到那些原本不是数字的行
    swapped_rows_mask = gen_as_numeric.isna()
    
    print(f"找到 {swapped_rows_mask.sum()} 条可能被置换的数据。")

    if swapped_rows_mask.sum() > 0:
        print("\n置换前的数据:")
        print(df_cleaned[swapped_rows_mask][['Name', 'Generation', 'Legendary']])

        # 对找到的行进行数据交换
        for index in df_cleaned[swapped_rows_mask].index:
            # 使用临时变量保存原始值
            original_gen = df_cleaned.loc[index, 'Generation']
            original_leg = df_cleaned.loc[index, 'Legendary']
            
            # 执行交换
            df_cleaned.loc[index, 'Generation'] = original_leg
            df_cleaned.loc[index, 'Legendary'] = original_gen

        print("\n置换后的数据:")
        print(df_cleaned.loc[swapped_rows_mask.index][['Name', 'Generation', 'Legendary']])
        print("\n成功修正 Generation 和 Legendary 的数据。")
    else:
        print("未发现需要修正的 Generation 和 Legendary 数据。")

except Exception as e:
    print(f"处理时发生错误: {e}")

找到 6 条可能被置换的数据。

置换前的数据:
                 Name Generation  Legendary
11          Blastoise      FALSE          1
32            Pikachu      FALSE          0
39   Nidoranâ?Poison      FALSE        NaN
408               NaN        NaN        NaN
771         Heliolisk  undefined      FALSE
806         undefined  undefined  undefined

置换后的数据:
                          Name Generation  Legendary
0                    Bulbasaur          1      FALSE
1                      Ivysaur          1      FALSE
2                     Venusaur          1      FALSE
3        VenusaurMega Venusaur          1      FALSE
4                   Charmander          1      FALSE
5                   Charmeleon          1      FALSE
6                    Charizard          1      FALSE
7    CharizardMega Charizard X          1      FALSE
8    CharizardMega Charizard Y          1      FALSE
9                     Squirtle          1      FALSE
10                   Wartortle          1      FALSE
11                   B

In [32]:
out_path = "D:\Pokemon_clean.csv"
df.to_csv(out_path, index=False, encoding="utf-8")
print("已保存：", out_path)

已保存： D:\Pokemon_clean.csv
